In [114]:
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency, chi2 
from scipy.stats import bootstrap

In [3]:
df = pd.read_csv('dating_data.csv') # считываем данные

In [4]:
df.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [5]:
# мне было удобнее работать без dummy-переменных, поэтому вернул их к привычному виду

group_map = {0: 'control', 1: 'test'}
df['group'] = df['group'].map(group_map) 
match_map = {0: 'not_match', 1: 'match'}
df['is_match'] = df['is_match'].map(match_map)

In [6]:
new_df = df

In [7]:
new_df.head()

,user_id_1,user_id_2,group,is_match
0,79,91,test,match
1,716,353,test,match
2,423,677,control,not_match
3,658,165,test,match
4,969,155,control,match


In [8]:
# просто для визуального удобства соединил две колонки в одну

new_df['pair_code'] = new_df['user_id_1'].astype(str) + new_df['user_id_2'].astype(str)

In [9]:
# удалил "визуальный шум" из рабочей таблицы

new_df = new_df.drop(columns = ['user_id_1', 'user_id_2'])

In [10]:
#  создаем таблицу сопряженности для хи-квадрата

pd.crosstab(new_df.group, new_df.is_match)

is_match,match,not_match
group,,
control,934,3858
test,3909,5813


Обе наши переменные - категориальные, а значит будем использовать хи-квадрат

- $H_0$: взаимосвязи между переменными нет 
- $H_1$: взаимосвязь есть

In [11]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(new_df.group, new_df.is_match))

In [12]:
stat, p # p-value меньше 0.05, что позволяет нам отклонить нулевую гипотезу и утверждать, что связь между переменными статистически значима

(618.6889141576198, 1.4418299163662586e-136)

In [13]:
# посчитаем отношение количества мэтчей к общему числу действий в каждой из групп и по результатам расчетов сделаем вывод
test_coeff = (3909/(3909+5813))
control_coeff = (934/(934+3858))
print(test_coeff)
print(control_coeff)

0.40207776177741206
0.19490818030050083


## Вывод:

Для того чтобы выяснить, улучшил ли новый алгоритм работу сервиса, я выбрал такую метрику как **MPA (match per action)**. 

В **тестовой** группе показатель MPS составил **0.40**, а в контрольной - **0.19**. Тест показал, что данные взаимосвязаны, а значит отличия можно считать значимыми. Показатель MPS в тестовой группе примерно в **2 раза больше**, чем в контрольной, а значит можно сделать вывод о том, что новый алгоритм **значимо улучшил показатель мэтчей** и, следовательно, работу сервиса 

In [142]:
df2 = pd.read_csv('dating_data.csv') # считываем данные

In [144]:
df2.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [148]:
# беру user_id_1, тк в user_id_2 это те же самые юзеры
# считаю количество действий и количество мэтчей по каждому пользователю

control_group = df2.query("group == '0'").groupby('user_id_1')\
                   .agg({'group':'count','is_match':'sum'}).reset_index()

In [149]:
# считаю долю мэтчей от всех действий пользователя

control_group['luck'] = (control_group['is_match']/control_group['group']).round(2)

In [134]:
control_group.head()

,user_id_1,group,is_match,luck
0,4,9,2,0.22
1,10,6,3,0.50
2,12,12,1,0.08
3,14,8,0,0.00
4,15,20,3,0.15


In [135]:
# беру user_id_1, тк в user_id_2 это те же самые юзеры
# считаю количество действий и количество мэтчей по каждому пользователю

test_group = df2.query("group == '1'").groupby('user_id_1')\
                   .agg({'group':'count','is_match':'sum'}).reset_index()

In [150]:
# считаю долю мэтчей от всех действий пользователя

test_group['luck'] = (test_group['is_match']/test_group['group']).round(2)

In [151]:
test_group.head()

,user_id_1,group,is_match,luck
0,1,24,11,0.46
1,2,16,7,0.44
2,3,16,5,0.31
3,5,22,13,0.59
4,6,22,9,0.41


Распределения переменных оказались крайне далеки от нормального и логарифмирование не помогло, вместо U-теста решил провести bootstrap анализ медиан распределений

In [152]:
# считаю 95% доверительный интервал для медианы контрольной группы

bootstrap((control_group.luck, ), np.median, method='percentile')

BootstrapResult(confidence_interval=ConfidenceInterval(low=0.17, high=0.2), standard_error=0.010317652836713324)

In [153]:
# считаю 95% доверительный интервал для медианы тестовой группы

bootstrap((test_group.luck, ), np.median, method='percentile')

BootstrapResult(confidence_interval=ConfidenceInterval(low=0.4, high=0.41), standard_error=0.005696317912684274)

Мы можем заметить, что доверительные интервалы медиан коэффициента мэтчей для каждого пользователя MatchByUser (MBU) не пересекаются, это говорит о том, что медианы статистически значимо различаются

In [140]:
median_diff = []

for i in range(10000):
    sample_data_control = control_group.sample(frac=1, replace=True)
    sample_median_control = sample_data_control.luck.median()
    
    
    sample_data_test = test_group.sample(frac=1, replace=True)
    sample_median_test = sample_data_test.luck.median()
    
    sample_median_diff = sample_median_test - sample_median_control
    median_diff.append(sample_median_diff)

In [154]:
# верхняя граница 95%-го ДИ для разницы медиан в разных группах

pd.Series(median_diff).quantile(0.975)

0.23999999999999996

In [155]:
# нижняя граница 95%-го ДИ для разницы медиан в разных группах

pd.Series(median_diff).quantile(0.025)

0.2

Для того чтобы окончательно убедиться в правильности своего решения рассчитал еще и доверительный интервал для разностей медиан в разных группах, он не пересекает 0 и положителен, это подтверждает гипотезу о том, что наша разница статистически значима и положительна.

Проведение и интерпретация результатов bootstrap анализа позволили сделать вывод о том, что внедрение нового алгоритма позволит увеличить показатель метрики MatchByUser (MBU) примерно в два раза